In [14]:
import numpy as np
import netCDF4 as nc
import gplearn
import gplearn.genetic

## Load mitgcm data

In [23]:
loadLoc = "/scratch/hcm7920/amb0/data/"

data = nc.Dataset(loadLoc+"state.nc")
eta  = data["Eta"]
uVel = data["U"]
vVel = data["V"]
wVel = data["W"]

## Set up data for gplearn
- Output:
    - \eta @ t, cell center
- Inputs:
    - \eta @ t-dt, cell center
    - u @ t,    e-w cell faces
    - v @ t,    n-s cell faces
    - u @ t-dt, e-w cell faces
    - v @ t-dt, n-s cell faces


### 9 inputs, 1 output

### Add data to input & output arrays

In [24]:
numInputs = 9

etaNow  = eta[1::10,2:-2:15,2:-2:15]
flatOutput = (etaNow).flatten()/etaNow.max()

input = np.zeros(shape=etaNow.shape+(numInputs,))
flatInput = np.zeros(shape=flatOutput.shape+(numInputs,))

# eta in past
input[:,:,:,0] = eta[0::10,2:-2:15,2:-2:15]/etaNow.max()

# u @ east west faces now
input[:,:,:,1] = uVel[1::10,0,2:-2:15,2:-2:15]/0.02
input[:,:,:,2] = uVel[1::10,0,2:-2:15,3:-2:15]/0.02

# v @ north south faces now
input[:,:,:,3] = vVel[1::10,0,2:-2:15,2:-2:15]/0.02
input[:,:,:,4] = vVel[1::10,0,3:-2:15,2:-2:15]/0.02

# u @ east west faces in past
input[:,:,:,5] = uVel[0::10,0,2:-2:15,2:-2:15]/0.02
input[:,:,:,6] = uVel[0::10,0,2:-2:15,3:-2:15]/0.02

# v @ north south faces in past
input[:,:,:,7] = vVel[0::10,0,2:-2:15,2:-2:15]/0.02
input[:,:,:,8] = vVel[0::10,0,3:-2:15,2:-2:15]/0.02

### Flatten first 3 dimensions

In [25]:
for ix in range(numInputs):
  flatInput[:,ix] = (input[:,:,:,ix]).flatten()

In [26]:
print(flatInput.shape,
      "\n",
      flatOutput.shape)

(28900, 9) 
 (28900,)


### Setup and run gplearn model

In [27]:
srModel = gplearn.genetic.SymbolicRegressor(
  population_size=10000,
  generations=100,
  p_crossover=0.4,
  p_subtree_mutation=0.3,
  p_hoist_mutation=0.1,
  p_point_mutation=0.1,
  max_samples=0.9,
  verbose=1,
  parsimony_coefficient=0.001,
  function_set=("add", "mul","sub","div",
               "sqrt","log","abs","inv","neg"))

srModel.fit(flatInput,flatOutput)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     9.58          205.133        3        0.0309264        0.0319574     18.63m
   1     6.72          3.23445        3        0.0308599        0.0325551     17.30m
   2     4.56          2.81976        1        0.0307771        0.0333011     15.14m
   3     2.38          2.69478        3        0.0307893        0.0331912     14.17m
   4     1.93          2.38507        1        0.0307536        0.0335126     13.13m
   5     1.93          5.11355        1        0.0307686         0.033377     13.01m
   6     1.91          2.74075        1        0.0307506        0.0335396     12.84m
   7     1.93          2.13233        1        0.0307486        0.0335572     12.74m
   8     1.91          2.47741        1        0.0307682        0.0333811  

SymbolicRegressor(function_set=('add', 'mul', 'sub', 'div', 'sqrt', 'log',
                                'abs', 'inv', 'neg'),
                  generations=100, max_samples=0.9, p_crossover=0.4,
                  p_hoist_mutation=0.1, p_point_mutation=0.1,
                  p_subtree_mutation=0.3, population_size=10000, verbose=1)

## Argh! It decided that it predicts best with just $\eta$
- Many attempts with different parameters yields the same result
- Expected I suppose, considering the snapshots are 1 day apart (vs 1 timestep apart)

### Once more, but this time without previous $\eta$
- Requires the model to use u,v

In [28]:
srModel = gplearn.genetic.SymbolicRegressor(
  population_size=5000,
  generations=50,
  p_crossover=0.4,
  p_subtree_mutation=0.3,
  p_hoist_mutation=0.1,
  p_point_mutation=0.1,
  max_samples=0.9,
  verbose=1,
  parsimony_coefficient=0.001,
  function_set=("add", "mul","sub","div",
               "sqrt","log","abs","inv","neg"))

srModel.fit(flatInput[:,1:],flatOutput)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     9.56          794.245        3         0.289575         0.296601      4.62m
   1     6.93           6.4685        6         0.289073         0.301123      4.36m
   2     5.36           6.8811        3         0.288497         0.306306      3.90m
   3     3.38          3.32341        4         0.288998           0.3018      3.31m
   4     2.56          10.3657        3         0.288939         0.302328      3.55m
   5     1.91          2.65482        1         0.289005         0.304025      3.01m
   6     2.02          3.18099        1         0.289218         0.302107      2.96m
   7     1.93          2.35738        1         0.289023         0.303862      2.90m
   8     1.94          3.12876        1         0.289256         0.301765  

SymbolicRegressor(function_set=('add', 'mul', 'sub', 'div', 'sqrt', 'log',
                                'abs', 'inv', 'neg'),
                  generations=50, max_samples=0.9, p_crossover=0.4,
                  p_hoist_mutation=0.1, p_point_mutation=0.1,
                  p_subtree_mutation=0.3, population_size=5000, verbose=1)

## Try again, predicting w instead of $\eta$
- Same input, different output

## Set up data for gplearn
- Output:
    - w @ t, cell center
- Inputs:
    - \eta @ t-dt, cell center
    - u @ t,    e-w cell faces
    - v @ t,    n-s cell faces
    - u @ t-dt, e-w cell faces
    - v @ t-dt, n-s cell faces


### 9 inputs, 1 output

In [31]:
wNow  = wVel[1::10,0,2:-2:15,2:-2:15]
flatOutput = (wNow).flatten()/wNow.max()

### Setup and run sr model again

In [32]:
srModel = gplearn.genetic.SymbolicRegressor(
  population_size=2000,
  generations=50,
  p_crossover=0.6,
  p_subtree_mutation=0.1,
  p_hoist_mutation=0.1,
  p_point_mutation=0.1,
  max_samples=0.9,
  verbose=1,
  parsimony_coefficient=0.001,
  function_set=("add","sub","div","mul",
               "sqrt","log","max","min"))

srModel.fit(flatInput,flatOutput)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    17.56          93651.6        4        0.0733942        0.0756871      2.30m
   1    10.23          2.02701        3        0.0732789        0.0767247      1.93m
   2     8.53          1.19773        3        0.0732182        0.0772715      1.74m
   3     3.22          1.54322        3        0.0731627        0.0777703      1.25m
   4     2.94          1.23447        3        0.0731169        0.0781829      1.21m
   5     1.73         0.788875        3        0.0732496        0.0769886      1.14m
   6     1.39         0.711939        7        0.0735848        0.0739719      1.10m
   7     1.35          1.27949        3        0.0737845        0.0721749      1.07m
   8     1.36          1.08398        3        0.0733653        0.0759477  

SymbolicRegressor(function_set=('add', 'sub', 'div', 'mul', 'sqrt', 'log',
                                'max', 'min'),
                  generations=50, max_samples=0.9, p_crossover=0.6,
                  p_hoist_mutation=0.1, p_point_mutation=0.1,
                  p_subtree_mutation=0.1, population_size=2000, verbose=1)

## Hmmm..